# This R notebook comtrains the code for API extraction

In [ ]:
library(googleway)
library(RCurl)
library(RJSONIO)
library(tibble)
library(dplyr)
library(httr)
library(tidyverse)
library(jsonlite)
library(stringr)
library(lubridate)
library(DataCombine)


In [1]:
#API key from google 
api_key <- 'KEY HERE'

In [2]:
#part of URL to be called 
plcUrl <- "https://maps.googleapis.com/maps/api/place/textsearch/json?query= "

### The following code calls the url and extracts the data store it into the a dataframe

In [ ]:
get_data<-function(api_key,page_token){
  #change query to hospital or schools
    query <-as.character(paste("Schools in",city))
  print(city)
  plcUrl <- "https://maps.googleapis.com/maps/api/place/textsearch/json?query= "
  #remove type=schools when extracting hospitals else it wont work
  strurl <- as.character(paste(plcUrl ,query,"&key=",api_key,"&type=school,primary_school,secondary_school",sep=""))
  
  
  data <- fromJSON(URLencode(strurl))
  
 
  #status =OK means there are data extracted.
  if (data$status == "OK"){
      #update the page token
    page_token <- data$next_page_token 
    df<-as.data.frame(data$results)
    
    lat <- split(data$results$geometry$location, seq(nrow(data$results$geometry$location)))
    df$geometry <-lat
    df$lat<-data$results$geometry$location$lat
    df$long<-data$results$geometry$location$lng
    df$city <- city
    
    openHours <- split(data$results$opening_hours, seq(nrow(data$results$opening_hours)))
    df$opening_hours <-openHours
    df$hours <- data$results$opening_hours
    
    plusCode <- split(data$results$plus_code, seq(nrow(data$results$plus_code)))
    df$plus_code <-plusCode
    print(nrow(loc_df))
    
    if(nrow(loc_df)==0){
      loc_df<<-df
      Sys.sleep(20)
      
    }else{
      loc_df <<-InsertRow(loc_df,NewRow =df )
      Sys.sleep(20)
    }
    page_token <- data$next_page_token 
    if(is.null(page_token)){
      return (loc_df)
    }else{
        #pass new pagetoken to get next page
    get_data(api_key,page_token)
    }
  }
  else {
    Sys.sleep(20)
    get_data(api_key,page_token)
  }
  
}


In [ ]:
#The first call gives a null value for the page_token
location_df <- get_data(api_key,"")
View(location_df)

In [ ]:
#remove the unwanted list data
data_extracted <-  location_df%>%dplyr::select(-geometry,-opening_hours,-plus_code,-types,-photos)


In [ ]:
#store each city with the file name exptention
file <- paste(as.character(paste("schools_", city, ".csv", sep="")))
write.csv(data_extracted, file)
